---
# Name of Method - Model 3
---
In this notebook, ...


## Results

---

### Importing necessary library

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier


### Read data file

In [2]:
churn_df = pd.read_excel('../data/churn_cleaned_featEng.xlsx')
churn_df

,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Value,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Internet Service_Fiber optic,Internet Service_No,...,Payment Method_Electronic check,Payment Method_Mailed check,Gender_Male,Phone Service_Yes,Multiple Lines_No phone service,Multiple Lines_Yes,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes
0,33.964131,118.272783,2,53.85,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
1,34.059281,118.307420,2,70.70,1,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
2,34.048013,118.293953,8,99.65,1,0,0,1,1,0,...,1,0,0,1,0,1,0,1,0,1
3,34.062125,118.315709,28,104.80,1,0,1,1,1,0,...,1,0,0,1,0,1,0,1,0,1
4,34.039224,118.266293,49,103.70,1,0,0,1,1,0,...,0,0,1,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,34.341737,116.539416,72,21.15,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,1,0
7039,34.667815,117.536183,24,84.80,0,0,1,1,0,0,...,0,1,1,1,0,1,0,1,0,1
7040,34.559882,115.637164,72,103.20,0,0,1,1,1,0,...,0,0,0,1,0,1,0,1,0,1
7041,34.167800,116.864330,11,29.60,0,0,1,1,0,0,...,1,0,0,0,1,0,0,0,0,0


---

<center>
    
## Preparing data

</center>

---

### Separate X and y features

In [3]:
# Seperate X and y features
X = churn_df.drop(columns=['Churn Value'])
y = churn_df['Churn Value']

### Split dataset (training/testing)

In [4]:
# Separating the dataset into a training dataset (70%) and testing+validation (30%) dataset
X_train, X_test_validation, y_train, y_test_validation = train_test_split(X, y, train_size=0.7, random_state=5)

# Separating the testing+valisation dataset into a testing dataset (15%) and a validation dataset (15%) 
X_val, X_test, y_val, y_test = train_test_split(X_test_validation, y_test_validation, test_size=0.5, random_state=5)

### Converting data subset to dataframe 

In [5]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### Scale X features

In [6]:
# Create instance of scaler
scaler = StandardScaler()

# Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

<center>
    
## Random Forest Model

</center>

---

### Find best parameters for model


### Run model on training dataset

In [7]:
# Create instance of model with default values
rfc = RandomForestClassifier(random_state=0)

# Fit  model
rfc.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=0)

### Investigate best model's predictive features

In [8]:
y_pred = rfc.predict(X_test_scaled)

print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 100 decision-trees : 0.8089


In [9]:
# Classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       783
           1       0.65      0.56      0.60       274

    accuracy                           0.81      1057
   macro avg       0.75      0.73      0.74      1057
weighted avg       0.80      0.81      0.80      1057



In [10]:
# Create instance of model witt n_estimators=200
rfc_200 = RandomForestClassifier(n_estimators=200, random_state=0)

# Fit model
rfc_200.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [11]:
y_pred_200 = rfc_200.predict(X_test_scaled)

print('Model accuracy score with 200 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_200)))

Model accuracy score with 200 decision-trees : 0.8117


Model accuracy score with 100 decision-trees : 0.7928

Model accuracy score with 200 decision-trees : 0.7379

## Selecting Important Features


In [12]:
# View feature scores

feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

Tenure Months                             0.173259
Monthly Charges                           0.138571
Latitude                                  0.121355
Longitude                                 0.119505
Internet Service_Fiber optic              0.045527
Payment Method_Electronic check           0.038983
Dependents_Yes                            0.035907
Contract_Two year                         0.034389
Contract_One year                         0.026466
Online Security_Yes                       0.023113
Gender_Male                               0.022691
Partner_Yes                               0.022000
Paperless Billing_Yes                     0.020619
Tech Support_Yes                          0.020408
Senior Citizen_Yes                        0.018516
Online Backup_Yes                         0.017909
Multiple Lines_Yes                        0.017634
Device Protection_Yes                     0.015504
Streaming Movies_Yes                      0.015199
Streaming TV_Yes               

In [18]:
# Separate X and y features
# Drop 'Phone Service_Yes','Multiple Lines_No phone service','Streaming TV_No internet service' as it's not important
churn_df = churn_df.drop(['Phone Service_Yes','Multiple Lines_No phone service','Streaming TV_No internet service'], axis=1)
X = churn_df.drop(columns=['Churn Value'])
y = churn_df['Churn Value']
# Separating the dataset into a training dataset (70%) and testing+validation (30%) dataset
X_train, X_test_validation, y_train, y_test_validation = train_test_split(X, y, train_size=0.7, random_state=5)

# Separating the testing+valisation dataset into a testing dataset (15%) and a validation dataset (15%) 
X_val, X_test, y_val, y_test = train_test_split(X_test_validation, y_test_validation, test_size=0.5, random_state=5)

X_train_scaled2 = scaler.fit_transform(X_train)
X_test_scaled2 = scaler.transform(X_test)

# Create instance of model
clf = RandomForestClassifier()

# Fit model
clf.fit(X_train_scaled2, y_train)

RandomForestClassifier()

In [19]:
y_pred_clf = clf.predict(X_test_scaled2)
print('Model accuracy score with three less features : {0:0.4f}'. format(accuracy_score(y_test, y_pred_clf))) 

Model accuracy score with one less feature : 0.8212


In [20]:
cm = confusion_matrix(y_test, y_pred_clf)

print(cm)

[[710  73]
 [116 158]]


In [21]:
print(classification_report(y_test, y_pred_clf))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       783
           1       0.68      0.58      0.63       274

    accuracy                           0.82      1057
   macro avg       0.77      0.74      0.75      1057
weighted avg       0.81      0.82      0.82      1057



### To have the optimized parameters and compare different values we applied GridSearchCV on RandomForest model.

In [17]:

# Create a pipeline with a scaler and Random Forest classifier
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),  # Scaling the data
    ('classifier', RandomForestClassifier(random_state=42))  # Random Forest classifier
])

# Define the parameter grid for Random Forest
param_grid_rf = {
    'classifier__n_estimators': [100, 200],  # Number of trees in the forest
    'classifier__max_depth': [None, 10, 20],       # Maximum depth of the trees
    'classifier__min_samples_split': [2, 5, 10],       # Minimum samples required to split an internal node
    'classifier__min_samples_leaf': [1, 2, 3],         # Minimum samples required to be at a leaf node
    'classifier__max_features': ['auto', 'sqrt'],  # Number of features to consider at each split
    'classifier__class_weight': ['balanced', None],    # Handling class imbalance
    'classifier__criterion': ['entropy']      # Function to measure the quality of a split
}

# Perform GridSearchCV to find the best hyperparameters
grid_search_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, cv=5, verbose=1)
grid_search_rf.fit(X_train_scaled, y_train)  # Fit using training data

# Output the best parameters and the corresponding score
print("Best parameters found: ", grid_search_rf.best_params_)
print("Best cross-validation score: ", grid_search_rf.best_score_)

# Optionally, you can also print the best model's performance on the test data
best_model_rf = grid_search_rf.best_estimator_
test_score_rf = best_model_rf.score(X_test_scaled, y_test)
print("Test score of the best model: ", test_score_rf)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


C:\Users\Masoum Saberi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Masoum Saberi\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Masoum Saberi\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Masoum Saberi\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 473, 

Best parameters found:  {'classifier__class_weight': None, 'classifier__criterion': 'entropy', 'classifier__max_depth': 10, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Best cross-validation score:  0.8040567951318458
Test score of the best model:  0.8145695364238411


---

<center>
    
## Validation

</center>

---

### Run model on testing dataset

### Model's validation

---

<center>
    
## Results

</center>

---

### Results